In [5]:
!pip install --quiet -U langchain langchain-community langchain-aws langchain-core pgvector

import os
import pandas as pd
import numpy as np
import json
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
import boto3
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import BedrockLLM
import time
import logging
import pandas as pd
from IPython.display import display, Markdown

print("Imported and installed dependencies!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.
Imported and installed dependencies!


In [6]:
import os
import json
import boto3
import pandas as pd
import ast

def get_bedrock_embeddings(input_text, model_id="amazon.titan-embed-text-v2:0", region_name="ca-central-1"):
    """Fetches text embeddings from AWS Bedrock."""
    bedrock = boto3.client(service_name='bedrock-runtime', region_name=region_name)
    
    body = json.dumps({
        "inputText": input_text,
        "dimensions": 1024,
        "normalize": True
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=model_id,
        accept="*/*",
        contentType="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body.get('embedding', [])

get_bedrock_embeddings("hello")




[-0.05515049770474434,
 0.04516850784420967,
 0.02720092423260212,
 0.006394712720066309,
 0.04142526164650917,
 -0.0359351672232151,
 0.009233341552317142,
 0.0314432717859745,
 0.017094159498810768,
 0.010793027468025684,
 -0.009794828481972218,
 0.03718291595578194,
 0.007611267734318972,
 -0.05415229871869087,
 -0.011104964651167393,
 -0.03543606773018837,
 -0.014972985722124577,
 -0.035685617476701736,
 0.04766400530934334,
 0.002916612895205617,
 -0.03393876925110817,
 0.016220735386013985,
 0.03393876925110817,
 -0.020587855949997902,
 -0.027700023725628853,
 -0.02607795037329197,
 -0.035186517983675,
 -0.03543606773018837,
 0.00948289129883051,
 -0.05714689567685127,
 0.04741445556282997,
 -0.0070497808046638966,
 -0.008235142566263676,
 -0.01397478673607111,
 0.0160959605127573,
 0.03793156519532204,
 0.019963981583714485,
 -0.02345767803490162,
 -0.012352713383734226,
 -0.01647028513252735,
 0.05440184846520424,
 -0.028199123218655586,
 -0.0061139692552387714,
 0.021086955443

In [7]:
def create_dynamodb_history_table(table_name: str) -> bool:
    """
    Create a DynamoDB table to store the session history if it doesn't already exist.

    Args:
    table_name (str): The name of the DynamoDB table to create.

    Returns:
    None
    
    If the table already exists, this function does nothing. Otherwise, it creates a 
    new table with a key schema based on 'SessionId'.
    """
    # Get the service resource and client.
    dynamodb_resource = boto3.resource("dynamodb")
    dynamodb_client = boto3.client("dynamodb")
    
    # Retrieve the list of tables that currently exist.
    existing_tables = []
    exclusive_start_table_name = None
    
    while True:
        if exclusive_start_table_name:
            response = dynamodb_client.list_tables(ExclusiveStartTableName=exclusive_start_table_name)
        else:
            response = dynamodb_client.list_tables()
        
        existing_tables.extend(response.get('TableNames', []))
        
        if 'LastEvaluatedTableName' in response:
            exclusive_start_table_name = response['LastEvaluatedTableName']
        else:
            break
    
    if table_name not in existing_tables:  # Create a new table if it doesn't exist.
        # Create the DynamoDB table.
        table = dynamodb_resource.create_table(
            TableName=table_name,
            KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
            AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
            BillingMode="PAY_PER_REQUEST",
        )
        
        # Wait until the table exists.
        table.meta.client.get_waiter("table_exists").wait(TableName=table_name)

create_dynamodb_history_table("test")

In [8]:
secrets_manager_client = boto3.client("secretsmanager")
ssm_client = boto3.client("ssm", region_name="ca-central-1")

def get_secret(secret_name, expect_json=True):
    global db_secret
    if db_secret is None:
        try:
            response = secrets_manager_client.get_secret_value(SecretId=secret_name)["SecretString"]
            db_secret = json.loads(response) if expect_json else response
        except json.JSONDecodeError as e:
            logger.error(f"Failed to decode JSON for secret {secret_name}: {e}")
            raise ValueError(f"Secret {secret_name} is not properly formatted as JSON.")
        except Exception as e:
            logger.error(f"Error fetching secret {secret_name}: {e}")
            raise
    return db_secret


def get_parameter(param_name, cached_var):
    """
    Fetch a parameter value from Systems Manager Parameter Store.
    """
    if cached_var is None:
        try:
            response = ssm_client.get_parameter(Name=param_name, WithDecryption=True)
            cached_var = response["Parameter"]["Value"]
        except Exception as e:
            logger.error(f"Error fetching parameter {param_name}: {e}")
            raise
    return cached_var

In [9]:
secrets_raw = secrets_manager_client.get_secret_value(SecretId='virt-care-Database-VCI/credentials/rdsDbCredential')["SecretString"]
secrets = json.loads(secrets_raw)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the GetSecretValue operation: Secrets Manager can't find the specified secret.

In [ ]:


# Set up basic logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Environment variables
DB_SECRET_NAME = "virt-care-Database-VCI/credentials/rdsDbCredential" #os.environ["SM_DB_CREDENTIALS"]
REGION = "ca-central-1" #os.environ["REGION"]
RDS_PROXY_ENDPOINT = secrets.get("host") #os.environ["RDS_PROXY_ENDPOINT"]
BEDROCK_LLM_PARAM =  "meta.llama3-8b-instruct-v1:0" #os.environ["BEDROCK_LLM_PARAM"]
EMBEDDING_MODEL_PARAM = "amazon.titan-embed-text-v2:0"#os.environ["EMBEDDING_MODEL_PARAM"]
TABLE_NAME_PARAM = "DynamoDB-Conversation-Table" #os.environ["TABLE_NAME_PARAM"]

# AWS Clients
secrets_manager_client = boto3.client("secretsmanager")
ssm_client = boto3.client("ssm", region_name=REGION)
bedrock_runtime = boto3.client("bedrock-runtime", region_name=REGION)

# Cached resources
connection = None
db_secret = None
BEDROCK_LLM_ID = None
EMBEDDING_MODEL_ID = None
TABLE_NAME = "test"

# Cached embeddings instance
embeddings = None



def initialize_constants():
    global BEDROCK_LLM_ID, EMBEDDING_MODEL_ID, TABLE_NAME, embeddings
    BEDROCK_LLM_ID = get_parameter(BEDROCK_LLM_PARAM, BEDROCK_LLM_ID)
    EMBEDDING_MODEL_ID = get_parameter(EMBEDDING_MODEL_PARAM, EMBEDDING_MODEL_ID)
    TABLE_NAME = get_parameter(TABLE_NAME_PARAM, TABLE_NAME)

    if embeddings is None:
        embeddings = BedrockEmbeddings(
            model_id=EMBEDDING_MODEL_ID,
            client=bedrock_runtime,
            region_name=REGION,
        )
    
    create_dynamodb_history_table(TABLE_NAME)

def connect_to_db():
    global connection
    if connection is None or connection.closed:
        try:
            secret = get_secret(DB_SECRET_NAME)
            connection_params = {
                'dbname': secret["dbname"],
                'user': secret["username"],
                'password': secret["password"],
                'host': RDS_PROXY_ENDPOINT,
                'port': secret["port"]
            }
            connection_string = " ".join([f"{key}={value}" for key, value in connection_params.items()])
            connection = psycopg2.connect(connection_string)
            logger.info("Connected to the database!")
        except Exception as e:
            logger.error(f"Failed to connect to database: {e}")
            if connection:
                connection.rollback()
                connection.close()
            raise
    return connection

In [ ]:


def get_system_prompt(case_id):
    connection = connect_to_db()
    if connection is None:
        logger.error("No database connection available.")
        return {
            "statusCode": 500,
            "body": json.dumps("Database connection failed.")
        }
    
    try:
        cur = connection.cursor()
        
        cur.execute("""
            SELECT system_prompt, case_title,case_type, law_type
            FROM "cases"
            WHERE case_id = %s;
        """, (case_id,))

        result = cur.fetchone()
        logger.info(f"Query result: {result}")
        system_prompt = result[0] if result else None

        cur.close()

        if system_prompt:
            logger.info(f"System prompt for simulation_group_id {case_id} found: {system_prompt}")
        else:
            logger.warning(f"No system prompt found for simulation_group_id {case_id}")

        return system_prompt

    except Exception as e:
        logger.error(f"Error fetching system prompt: {e}")
        if cur:
            cur.close()
        connection.rollback()
        return None

In [ ]:
!pip install langchain_postgres

In [ ]:
import logging
from typing import Optional

import psycopg2
from langchain_aws import BedrockEmbeddings
from langchain_postgres import PGVector

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_vectorstore(
    collection_name: str, 
    embeddings: BedrockEmbeddings, 
    dbname: str, 
    user: str, 
    password: str, 
    host: str, 
    port: int
) -> Optional[PGVector]:
    """
    Initialize and return a PGVector instance.
    
    Args:
    collection_name (str): The name of the collection.
    embeddings (BedrockEmbeddings): The embeddings instance.
    dbname (str): The name of the database.
    user (str): The database user.
    password (str): The database password.
    host (str): The database host.
    port (int): The database port.
    
    Returns:
    Optional[PGVector]: The initialized PGVector instance, or None if an error occurred.
    """
    try:
        connection_string = (
            f"postgresql+psycopg://{user}:{password}@{host}:{port}/{dbname}"
        )

        logger.info("Initializing the VectorStore")
        vectorstore = PGVector(
            embeddings=embeddings,
            collection_name=collection_name,
            connection=connection_string,
            use_jsonb=True
        )

        logger.info("VectorStore initialized")
        return vectorstore, connection_string

    except Exception as e:
        logger.error(f"Error initializing vector store: {e}")
        return None

In [ ]:
from typing import Dict

from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

def get_vectorstore_retriever(
    llm,
    vectorstore_config_dict: Dict[str, str],
    embeddings#: BedrockEmbeddings
) -> VectorStoreRetriever:
    """
    Retrieve the vectorstore and return the history-aware retriever object.

    Args:
    llm: The language model instance used to generate the response.
    vectorstore_config_dict (Dict[str, str]): The configuration dictionary for the vectorstore, including parameters like collection name, database name, user, password, host, and port.
    embeddings (BedrockEmbeddings): The embeddings instance used to process the documents.

    Returns:
    VectorStoreRetriever: A history-aware retriever instance.
    """
    vectorstore, _ = get_vectorstore(
        collection_name=vectorstore_config_dict['collection_name'],
        embeddings=embeddings,
        dbname=vectorstore_config_dict['dbname'],
        user=vectorstore_config_dict['user'],
        password=vectorstore_config_dict['password'],
        host=vectorstore_config_dict['host'],
        port=int(vectorstore_config_dict['port'])
    )

    retriever = vectorstore.as_retriever()

    # Contextualize question and create history-aware retriever
    contextualize_q_system_prompt = (
        """Given a chat history and the latest user question 
        which might reference context in the chat history, 
        formulate a standalone response which can be understood 
        without the chat history, but may make references to past messages as well.
        Just reformulate it if needed and otherwise return it as is."""
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    return history_aware_retriever

In [ ]:
case_id = '550e8400-e29b-41d4-a716-546655480010'

In [ ]:
logger.info("Retrieving vectorstore config.")
db_secret = get_secret(DB_SECRET_NAME)
vectorstore_config_dict = {
            'collection_name': case_id,
            'dbname': db_secret["dbname"],
            'user': db_secret["username"],
            'password': db_secret["password"],
            'host': RDS_PROXY_ENDPOINT,
            'port': db_secret["port"]
    }

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

def get_response(query: str, llm: ChatBedrock, history_aware_retriever, table_name: str, session_id: str, system_prompt: str) -> dict:
    """
    Generates a response to a query using the LLM and a history-aware retriever to store context.

    Args:
    - query (str): The student's query string for which a response is needed.
    - llm (ChatBedrock): The language model instance used to generate the response.
    - history_aware_retriever: The history-aware retriever instance.
    - table_name (str): The DynamoDB table name used to store and retrieve chat history.
    - session_id (str): The unique identifier for the chat session.
    - system_prompt (str): The system prompt to instruct the model.

    Returns:
    - dict: A dictionary containing the generated response.
    """

    # Set up the memory with conversation buffer
    memory = ConversationBufferMemory(memory_key="messages", return_messages=True)

    # Set up the prompt template
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}\n\nContext: {context}")
    ])

    # Create a retrieval chain with the history-aware retriever
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # Create the conversation chain
    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory,
        retriever=rag_chain
    )

    # Use the conversation chain to generate the response
    response = conversation_chain.run(query)

    return {"answer": response}


In [12]:
# Defining Constants
LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
MISTRAL_7B = "mistral.mistral-7b-instruct-v0:2"
MISTRAL_LARGE = "mistral.mistral-large-2402-v1:0"
LLAMA_3_1_8B = "meta.llama3-1-8b-instruct-v1:0"
LLAMA_3_1_70B = "meta.llama3-1-70b-instruct-v1:0"

In [ ]:
import time
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

case_memory_store = {}

def get_memory(case_id):
    if case_id not in case_memory_store:
        case_memory_store[case_id] = ConversationBufferMemory(return_messages=False, max_length=3)  # Limits history
    return case_memory_store[case_id]

def answer_prompt(user_prompt, number_of_docs, case_id):
    total_start_time = time.time()
    answer_start_time = time.time()

    embedding = get_bedrock_embeddings(user_prompt)
    llm = BedrockLLM(model_id=LLAMA_3_8B)
    memory = get_memory(case_id)

    # Get system prompt
    system_prompt = get_system_prompt(case_id)
    print(system_prompt)

    # ✅ Updated Prompt Template to enforce single response
    custom_prompt = PromptTemplate.from_template(
        "{system_prompt}\n\nConversation History:\n{history}\n\nUser: {input}\nAssistant: (Provide a single response and do not continue the conversation. End your response clearly.)"
    )

    # Create ConversationChain with the correct prompt format
    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=custom_prompt.partial(system_prompt=system_prompt)  # Inject system prompt
    )

    # Get response
    answer = conversation_chain.predict(input=user_prompt).strip()  # Ensures clean output

    answer_duration = time.time() - answer_start_time
    total_duration = time.time() - total_start_time

    return {
        "answer": answer,
        "answer_time": answer_duration,
        "total_time": total_duration
    }

def neat_print(response):
    print(f"Answer: {response['answer']}\n")
    print("_________________________________________________________________________________________________________")
    print(f"answer_time: {response['answer_time']}\n")
    print(f"total_time: {response['total_time']}\n")

case_id = '550e8400-e29b-41d4-a716-546655480010'
response = answer_prompt("My client (from Toronto) is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5, case_id)
response2 = answer_prompt("Where does my client live again?", 5, case_id)

neat_print(response)
neat_print(response2)


In [ ]:
import time
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

case_memory_store = {}

def get_memory(case_id):
    if case_id not in case_memory_store:
        case_memory_store[case_id] = ConversationBufferMemory(return_messages=False, max_length=3)  # Limits history
    return case_memory_store[case_id]

def answer_prompt(user_prompt, number_of_docs, case_id):
    total_start_time = time.time()
    answer_start_time = time.time()

    embedding = get_bedrock_embeddings(user_prompt)
    llm = BedrockLLM(model_id=LLAMA_3_8B)
    memory = get_memory(case_id)

    # Get system prompt
    system_prompt = get_system_prompt(case_id)
    print(system_prompt)

    # ✅ Updated Prompt Template to enforce single response
    custom_prompt = PromptTemplate.from_template(
        "{system_prompt}\n\nConversation History:\n{history}\n\nUser: {input}\nAssistant: (Provide a single response and do not continue the conversation. End your response clearly.)"
    )

    # Create ConversationChain with the correct prompt format
    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=custom_prompt.partial(system_prompt=system_prompt)  # Inject system prompt
    )

    # Get response
    answer = conversation_chain.predict(input=user_prompt).strip()  # Ensures clean output

    answer_duration = time.time() - answer_start_time
    total_duration = time.time() - total_start_time

    return {
        "answer": answer,
        "answer_time": answer_duration,
        "total_time": total_duration
    }

def neat_print(response):
    print(f"Answer: {response['answer']}\n")
    print("_________________________________________________________________________________________________________")
    print(f"answer_time: {response['answer_time']}\n")
    print(f"total_time: {response['total_time']}\n")

case_id = '550e8400-e29b-41d4-a716-546655480010'
response = answer_prompt("My client (from Toronto) is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5, case_id)
response2 = answer_prompt("Where does my client live again?", 5, case_id)

neat_print(response)
neat_print(response2)

In [ ]:
# import time
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationChain

# # Dictionary to store memory per case ID
# case_memory_store = {}

# def get_memory(case_id):
#     """Retrieve memory for a given case ID, creating one if it doesn't exist."""
#     if case_id not in case_memory_store:
#         case_memory_store[case_id] = ConversationBufferMemory(return_messages=True)
#     return case_memory_store[case_id]

# def clear_memory(case_id):
#     if case_id in case_memory_store:
#         case_memory_store[case_id] = ConversationBufferMemory(return_messages=True)
#         return case_memory_store[case_id]

# def answer_prompt(user_prompt, number_of_docs, case_id):
#     # Record the start times
#     total_start_time = time.time()
#     answer_start_time = time.time()

#     # Initialize the Bedrock Embeddings model
#     embedding = get_bedrock_embeddings(user_prompt)

#     # Get the LLM we want to invoke
#     llm = BedrockLLM(model_id=LLAMA_3_8B)
    
#     # Retrieve memory for the specific case ID
#     memory = get_memory(case_id)

#     # Create the conversation chain with LLM and memory
#     conversation_chain = ConversationChain(
#         llm=llm,
#         memory=memory
#     )

#     # Get assistant's response using conversation chain
#     answer = conversation_chain.predict(input=user_prompt)

#     # Record the end time and find the duration of the answer only
#     answer_end_time = time.time()
#     answer_duration = answer_end_time - answer_start_time

#     # Record the total processing time
#     total_end_time = time.time()
#     total_duration = total_end_time - total_start_time

#     return {
#         "answer": answer,
#         "answer_time": answer_duration,
#         "total_time": total_duration
#     }

# # Neatly prints dictionary returned by answer_prompt
# def neat_print(response):
#     print(f"Answer: {response['answer']}\n")
#     print("_________________________________________________________________________________________________________")
#     print(f"answer_time: {response['answer_time']}\n")
#     print(f"total_time: {response['total_time']}\n")

# # Example usage
# case_id = '550e8410-e29b-41d4-a716-447755440011'
# response = answer_prompt("My client (from Toronto) is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5, case_id)

# response2 = answer_prompt("Where does my client live again?", 5, case_id)

# neat_print(response)
# neat_print(response2)


In [ ]:
import time
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Dictionary to store memory per case ID
case_memory_store = {}

def get_memory(case_id):
    """Retrieve memory for a given case ID, creating one if it doesn't exist."""
    if case_id not in case_memory_store:
        case_memory_store[case_id] = ConversationBufferMemory(return_messages=True)
    return case_memory_store[case_id]

def clear_memory(case_id):
    if case_id in case_memory_store:
        case_memory_store[case_id] = ConversationBufferMemory(return_messages=True)
        return case_memory_store[case_id]

def answer_prompt(user_prompt, number_of_docs, case_id):
    # Record the start times
    total_start_time = time.time()
    answer_start_time = time.time()

    # Initialize the Bedrock Embeddings model
    embedding = get_bedrock_embeddings(user_prompt)

    # Get the LLM we want to invoke
    llm = BedrockLLM(model_id=LLAMA_3_70B)
    
    # Retrieve memory for the specific case ID
    memory = get_memory(case_id)

    # Create the conversation chain with LLM and memory
    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory
    )

    # Get assistant's response using conversation chain
    answer = conversation_chain.predict(input=user_prompt)

    # Record the end time and find the duration of the answer only
    answer_end_time = time.time()
    answer_duration = answer_end_time - answer_start_time

    # Record the total processing time
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    return {
        "answer": answer,
        "answer_time": answer_duration,
        "total_time": total_duration
    }

# Neatly prints dictionary returned by answer_prompt
def neat_print(response):
    print(f"Answer: {response['answer']}\n")
    print("_________________________________________________________________________________________________________")
    print(f"answer_time: {response['answer_time']}\n")
    print(f"total_time: {response['total_time']}\n")

# Example usage
case_id = '550e8410-e29b-41d4-a716-447755440011'


In [ ]:
# response = answer_prompt("My client (from Toronto) is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5, case_id)

#response2 = answer_prompt("Where does my client live again?", 5, case_id)

response_new = answer_prompt("Where do I live", 5, case_id)

neat_print(response_new)

In [15]:
def answer_prompt(user_prompt, number_of_docs):

    # Record the start times
    total_start_time = time.time()
    answer_start_time = time.time()

    # Initialize the Bedrock Embeddings model
    # embeddings = BedrockEmbeddings()
    embedding = get_bedrock_embeddings(user_prompt)

    # docs = get_combined_docs(embedding, number_of_docs)

    # divided_docs = split_docs(docs)
    # print(len(divided_docs["docs"]))

    # documents = format_docs(divided_docs["docs"])

    # Get the LLM we want to invoke
    llm = BedrockLLM(
                        model_id=LLAMA_3_70B
                    )
    

    case_examples = '''Our hope is that an AI tool used by a student in these scenarios would not attempt to “solve” the issue, as legal matters have infinitely possible outcomes which can be based on many criteria including the personal circumstances of the client.  It would be great however if the tool could provide the student with insights about the legal and factual issues which may be engaged in these circumstances.  This would then help the students think about what legal issues to further research and what factual issues they should be investigating.


Case 1 : 


The first scenario is based more on the student having had at least one interview with the client : 


Accused is charged with assault causing bodily harm contrary to section 267(b) of the Criminal Code.  Accused was having an argument in a bar with the Victim, and the Victim is claiming that the Accused punched him repeatedly in an unprovoked attack.  Accused versions of events is that he was fairly intoxicated but still had a good sense of what was going on.  Accused remembers that he and Victim argued and then Victim challenged him to a fight.  Accused says he started walking away.  Accused then says Victim then pushed Accused from behind and Accused says he then turned around and punched Victim in the stomach twice to defend himself.


LLM should: 


Hopefully the tool can gather information which sets out the “essential elements” of proving the offence of assault causing bodily harm:  
application of force, 
intent to apply force, 
victim not consenting to force, 
and that harm that is more than trifling
 
Great additional insights provided by the tool would be things like : 
 
-assault is an included offence of assault causing bodily harm
 
-the scenario above raises potential defence of self-defence and consent (and maybe set out the requirements of those defences)
 
-that the intoxication is not likely a relevant issue
 
-that there are critical factual issues in this case in terms of who started the physical altercation and the level of force used by the accused in his response
 
By letting the student know about the legal issues, it would likely help the students assess both the case and the factual issues which are relevant.  Even if it just provided basic legal frameworks the students should be looking at for this offence that would be helpful.


Case 2 : 


The second one is a scenario where an intake person at the clinic has some basic information but no details yet : 


Client lives in Vancouver BC and had family law issues.  She and her husband were married for two years and have one child.  She and the child recently left her husband because he was being abusive to her and their child.  She is seeking some sort of restraining order against her husband to protect her and her child and need some sort of child support because she is not working.


LLM should:  


In terms of the second scenario, I think we would be looking for a tool to provide some broader information, such as:
 
emergency court applications which are available for a person in these circumstances
 
the basic legal rights of the client and child in these circumstances and
 
maybe even some community resources able to assist in these circumstances'''
    # system_prompt = get_system_prompt(case_id)

    # history_aware_retriever = get_vectorstore_retriever(
    #         llm=llm,
    #         vectorstore_config_dict=vectorstore_config_dict,
    #         embeddings=embeddings
    #     )



    s3 = boto3.client("s3")
    bucket_name = "cit-api-citapidataingestionbucket687917c5-mgr4azmebcps"
    file_key = "system_prompt.txt"
    
    # Retrieve latest prompt
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    retrieved_prompt = response["Body"].read().decode("utf-8")

 
    # system_prompt = "You are a helpful UBC student advising assistant who answers with kindness while being concise. If the question does not relate to UBC, respond with 'IDK.'"
    # system_prompt = """You are a helpful UBC student advising assistant. 
    #                    Using the documents given to you, consicely answer the user's prompt with kindness. 
    #                    If the question does not relate to UBC, respond with 'IDK.'"""

    if llm.model_id == LLAMA_3_8B or llm.model_id == LLAMA_3_70B or llm.model_id == LLAMA_3_1_8B or llm.model_id == LLAMA_3_1_70B:
        prompt = f'''{retrieved_prompt}

    Case date: N/A'''
    else:
        prompt = f"""{system_prompt}. Provide your answer as if you are talking to a student.
            Here is the question: {user_prompt}
            """

    
    answer = llm.invoke(prompt)

    # Record the end time and find duration of answer only
    answer_end_time = time.time()
    answer_duration = answer_end_time - answer_start_time

    # check_docs = check_if_documents_relates(divided_docs["docs"], user_prompt, llm)
    # check_additional_docs = check_if_documents_relates(divided_docs["removed_docs"], user_prompt, llm)

    # Record the end time and find duration of the total time of checking over each document
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    return {"answer": answer,
            # "docs": check_docs,
            # "additional_docs": check_additional_docs,
            "answer_time": answer_duration, "total_time": total_duration}

# Neatly prints dictionary returned by answer_prompt
def neat_print(response):
    print(f"{response['answer']}\n")

#response = answer_prompt("My client is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5)
# response = answer_prompt("Accused is charged with assault causing bodily harm contrary to section 267(b) of the Criminal Code.  Accused was having an argument in a bar with the Victim, and the Victim is claiming that the Accused punched him repeatedly in an unprovoked attack.  Accused versions of events is that he was fairly intoxicated but still had a good sense of what was going on.  Accused remembers that he and Victim argued and then Victim challenged him to a fight.  Accused says he started walking away.  Accused then says Victim then pushed Accused from behind and Accused says he then turned around and punched Victim in the stomach twice to defend himself.", 5)

# neat_print(response)

In [19]:
assault_case_details = """Accused is charged with assault causing bodily harm contrary to section 267(b) of the Criminal Code.  
Accused was having an argument in a bar with the Victim, and the Victim is claiming that the Accused punched him repeatedly in an unprovoked attack.  
Accused versions of events is that he was fairly intoxicated but still had a good sense of what was going on.  
Accused remembers that he and Victim argued and then Victim challenged him to a fight.  
Accused says he started walking away.  
Accused then says Victim then pushed Accused from behind and Accused says he then turned around and punched Victim in the stomach twice to defend himself.
"""

assault_case_response = answer_prompt(assault_case_details, 5)

neat_print(assault_case_response)




You are helping me, the law student, analyze this case.



In [278]:
restraining_order_case_details = """
                                Client lives in Vancouver, BC, and has family
                                law issues. She and her husband were married
                                for two years and have one child. She and the
                                child recently left her husband because he was
                                being abusive to her and their child. She is
                                seeking some sort of restraining order against
                                her husband to protect herself and her child
                                and needs some form of child support because
                                she is not working.
                                """

restraining_order_case_response = answer_prompt(restraining_order_case_details, 5)

neat_print(restraining_order_case_response)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:langchain_aws.llms.bedrock:Using Bedrock Invoke API to generate response




                Based on the information provided, it appears that the client is seeking a restraining order and child support due to domestic violence. In British Columbia, the client may be eligible for an Emergency Protection Order (EPO) or a Restraining Order (RO) under the Family Law Act. An EPO is a temporary order that can be obtained quickly, usually within 24 hours, to provide immediate protection for the client and her child. An RO is a longer-term order that can provide ongoing protection and support.

                In terms of child support, the client may be eligible for support under the Family Maintenance Enforcement Program (FMEP) or through a court order. The client should also be aware that she may be eligible for other forms of support, such as spousal support, and that she should seek legal advice to determine her options.

                Additionally, the client may want to consider seeking support from local community resources, such as the Vancouver Women's 

In [252]:
case_details = """Accused is charged with assault causing bodily harm contrary to section 267(b) of the Criminal Code.  
Accused was having an argument in a bar with the Victim, and the Victim is claiming that the Accused punched him repeatedly in an unprovoked attack.  
Accused versions of events is that he was fairly intoxicated but still had a good sense of what was going on.  
Accused remembers that he and Victim argued and then Victim challenged him to a fight.  
Accused says he started walking away.  
Accused then says Victim then pushed Accused from behind and Accused says he then turned around and punched Victim in the stomach twice to defend himself."""

In [198]:
print(f"TABLE_NAME: {TABLE_NAME}")

TABLE_NAME: test


In [130]:
response3 = answer_prompt("What was the third possible question you said I could ask my client?", 5, case_id)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:langchain_aws.llms.bedrock:Using Bedrock Invoke API to generate response


In [131]:
response3

{'answer': ' Ah, the third possible question I suggested your client could ask themselves is "Consider a separation agreement". This can help outline the terms of the separation, including child custody, child support, and spousal support, which can reduce uncertainty and potential disputes.',
 'answer_time': 0.7696671485900879,
 'total_time': 0.7696688175201416}

In [202]:
sample_output_assault = """ Hi there, I'd be happy to help you analyze this case. Based on the information provided, it appears that the accused is charged with assault causing bodily harm under section 267(b) of the Criminal Code. 
                         To prove the offence of assault causing bodily harm, the prosecution must establish the essential elements, which include: 
                         1. The application of force;
                         2. The intent to apply force;
                         3. The victim did not consent to the force;
                         4. And that the harm caused was more than trifling. 
                         In this case, the victim claims that the accused punched him repeatedly in an unprovoked attack, which may suggest that the accused did not have a reasonable belief in the need to defend himself. 
                         However, the accused's version of events suggests that he was pushed from behind by the victim and that he punched the victim in self-defense. 
                         This raises potential defences of self-defence and consent, which the accused may be able to argue. 
                         It's also important to note that the accused's intoxication may not be a relevant issue in this case. 
                         There are also critical factual issues in this case, such as who started the physical altercation and the level of force used by the accused in his response. 
                         Some potential follow-up questions you could ask the client to clarify the facts include:
                         1. Can you describe the argument that occurred before the physical altercation?
                         2. Did you see the victim push you from behind before you punched him?
                         3. How did you feel after being pushed from behind? Did you feel threatened or scared?
                         4. Did you try to de-escalate the situation before punching the victim?
                         5. Can you describe the level of force you used when punching the victim?
                         I hope this helps! Let me know if you have any other questions or if there's anything else I can help with."""

In [204]:
print(sample_output)

 Hi there, I'd be happy to help you analyze this case. Based on the information provided, it appears that the accused is charged with assault causing bodily harm under section 267(b) of the Criminal Code. 
                 To prove the offence of assault causing bodily harm, the prosecution must establish the essential elements, which include: 
                 1. The application of force;
                 2. The intent to apply force;
                 3. The victim did not consent to the force;
                 4. And that the harm caused was more than trifling. 
                 In this case, the victim claims that the accused punched him repeatedly in an unprovoked attack, which may suggest that the accused did not have a reasonable belief in the need to defend himself. 
                 However, the accused's version of events suggests that he was pushed from behind by the victim and that he punched the victim in self-defense. 
                 This raises potential defences of sel

In [220]:
sample_output_restraining = """
                            Hi! Thank you for sharing this information with me. It sounds like a very 
                            serious and sensitive situation.
                            
                            Based on the information you've provided, it seems that the client is seeking 
                            a restraining order and child support due to domestic violence. In British 
                            Columbia, the client may be eligible for an Emergency Protection Order (EPO) 
                            or a Restraining Order (RO) under the Family Law Act.
                            
                            An EPO is a court order that can be obtained quickly, usually within 24 hours, 
                            to provide immediate protection for the client and her child. A RO, on the 
                            other hand, is a longer-term order that can provide ongoing protection and 
                            support for the client and her child.
                            
                            In terms of child support, the client may be eligible for support under the 
                            Family Maintenance Enforcement Program (FMEP) or through a court order. It's 
                            also important to note that the client may be eligible for other forms of 
                            support, such as counseling or legal aid, to help her and her child navigate 
                            this difficult situation.
                            
                            What would you like to know more about or discuss further? Are there any 
                            specific concerns or questions the client has about the restraining order or 
                            child support process? Please let me know and I'll do my best to provide more 
                            information and guidance.
                            """


In [222]:
print(sample_output_restraining)


                            Hi! Thank you for sharing this information with me. It sounds like a very 
                            serious and sensitive situation.
                            
                            Based on the information you've provided, it seems that the client is seeking 
                            a restraining order and child support due to domestic violence. In British 
                            Columbia, the client may be eligible for an Emergency Protection Order (EPO) 
                            or a Restraining Order (RO) under the Family Law Act.
                            
                            An EPO is a court order that can be obtained quickly, usually within 24 hours, 
                            to provide immediate protection for the client and her child. A RO, on the 
                            other hand, is a longer-term order that can provide ongoing protection and 
                            support for the client and her chi

In [22]:
!pip install lm-format-enforcer

In [23]:
!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 19.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 35.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 22.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 49.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 83.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 52.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 44.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
!pip install deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.0/739.0 kB 54.0 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 8.3.4
    Uninstalling pytest-8.3.4:
      Successfully uninstalled pytest-8.3.4
  Attempting uninstall: pkginfo
    Found existing installation: pkginfo 1.12.0
    Uninstalling pkginfo-1.12.0:
      Successfully uninstalled pkginfo-1.12.0
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uni

In [31]:
import json
import transformers
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
# from lmformatenforcer.integrations.transformers import (
#     build_transformers_prefix_allowed_tokens_fn,
# )

from deepeval.models import DeepEvalBaseLLM


class CustomLlama3_70B(DeepEvalBaseLLM):
    ...

    def __init__(self):
        llm = BedrockLLM(model_id=LLAMA_3_70B)        
        self.model = llm

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        # Get the LLM we want to invoke
        llm = self.load_model()


        llm = BedrockLLM(
            model_id=LLAMA_3_8B,
            streaming=True
                    )
    

        s3 = boto3.client("s3")
        bucket_name = "cit-api-citapidataingestionbucket687917c5-mgr4azmebcps"
        file_key = "system_prompt.txt"
        
        # Retrieve latest prompt
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        retrieved_prompt = response["Body"].read().decode("utf-8")
    
     
        # system_prompt = "You are a helpful UBC student advising assistant who answers with kindness while being concise. If the question does not relate to UBC, respond with 'IDK.'"
        # system_prompt = """You are a helpful UBC student advising assistant. 
        #                    Using the documents given to you, consicely answer the user's prompt with kindness. 
        #                    If the question does not relate to UBC, respond with 'IDK.'"""
    
        if llm.model_id == LLAMA_3_8B or llm.model_id == LLAMA_3_70B or llm.model_id == LLAMA_3_1_8B or llm.model_id == LLAMA_3_1_70B:
            prompt = f'''{retrieved_prompt}
    
        Case date: N/A'''
        else:
            prompt = f"""{system_prompt}. Provide your answer as if you are talking to a student.
                Here is the question: {user_prompt}
                """
            
        # Retrieve memory for the specific case ID
            
        # Get assistant's response using conversation chain
        answer = llm.invoke(input=prompt)
        
        # Create parser required for JSON confinement using lmformatenforcer
        #parser = JsonSchemaParser(schema.model_json_schema())
        # prefix_function = build_transformers_prefix_allowed_tokens_fn(
        #     pipeline.tokenizer, parser
        # )
        
        # Output and load valid JSON
        # output_dict = pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        # output = output_dict[0]["generated_text"][len(prompt) :]
        # json_result = json.loads(output)
        
        # Return valid JSON object according to the schema DeepEval supplied
        return answer # schema(**json_result)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Llama 3.1 70B"

    def load_model(self):
        return self.model

In [32]:
testmodel = CustomLlama3_70B()

In [34]:
# response_eval = testmodel.generate("My client was locked in a store overnight, and wants to sue for negligence. This was in Alberta, Calgary actually. What part of the law should I look to")
response_eval = testmodel.generate("Do an initial analysis on the case for me. what sections of the law are relevant?")

print(response_eval)


        Client's age: 25
        Client's occupation: Student
        Client's injuries: Sprained ankle, bruised knee
        Client's medical expenses: $1,000
        Client's lost wages: $2,000
        Client's other expenses: $500
        Client's total damages: $3,500
        Defendant's (McDonald's) insurance coverage: $1,000,000
        Defendant's (Apple) involvement: Apple owns McDonald's in Manitoba, but the client slipped and fell in a McDonald's that is not owned by Apple. Apple has no direct connection to the incident.

Please provide legal context and analysis, relevant issues, possible strategies to defend the client, and other important details in a structured natural language response. Also, provide possible follow-up questions for me, the law student, to ask the client to help progress the case more after your initial analysis.

Please keep in mind that this is a hypothetical case and not a real one.
